In [20]:
import numpy as np
import pandas as pd
from gurobipy import * 
import gurobipy as gp

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/iamzhenxp/E4418-Transportation-Anlytics-Logistic/FInal-project/FInal%20Project/Scooter%20Data.csv', encoding='unicode_escape')
map = pd.read_csv('https://raw.githubusercontent.com/iamzhenxp/E4418-Transportation-Anlytics-Logistic/FInal-project/FInal%20Project/CenterlineID.csv', encoding='unicode_escape')

In [3]:
##data.columns=['tripid','tripduration',]
data.head()

,ï»¿ObjectId,TripID,TripDuration,TripDistance,StartTime,EndTime,StartCenterlineID,StartCenterlineType,EndCenterlineID,EndCenterlineType
0,1,1256,289,1453,2021/04/29 20:00:00+00,2021/04/29 20:00:00+00,16644.0,street,15359.0,street
1,2,1336,490,1561,2021/04/29 20:00:00+00,2021/04/29 20:00:00+00,16638.0,street,26004.0,street
2,3,1344,246,96,2021/04/29 20:00:00+00,2021/04/29 20:00:00+00,16641.0,street,16641.0,street
3,4,1361,3900,5101,2021/04/29 20:00:00+00,2021/04/29 21:00:00+00,20562.0,street,15844.0,street
4,5,1357,734,3653,2021/04/29 20:29:59+00,2021/04/29 20:29:59+00,17095.0,street,NaN,trail


In [4]:
map.head()

,ï»¿OBJECTID,L_F_ADD,R_F_ADD,L_T_ADD,R_T_ADD,STREETALL,F_CLASS,ALT_NAM1,ALT_NAM2,ZIP5_L,...,F_XSTREET_1,F_XSTREET_2,F_XSTREET_3,T_XSTREET_1,T_XSTREET_2,T_XSTREET_3,GlobalID,FromLanes,ToLanes,Shape__Length
0,1,3000,3001,3098,3099,39TH ST E,A40,NaN,NaN,55406,...,30TH AVE S,NaN,NaN,NOKOMIS AVE S,NaN,NaN,29744cc9-5dea-4a48-a068-2daccb8ca02b,1,1.0,138.028160
1,2,601,600,613,612,SHERIDAN AVE N,A40,NaN,NaN,55411,...,OLSON MEMORIAL HWY,NaN,NaN,7TH AVE N,NaN,NaN,bc64c65c-369f-49e9-9cfc-c46b5aedb84d,1,1.0,24.108303
2,3,1998,1999,1900,1901,57TH ST W,A40,NaN,NaN,55419,...,MORGAN AVE S,NaN,NaN,LOGAN AVE S,NaN,NaN,0b2b1962-c712-4854-8b5e-9430e8009df5,1,1.0,142.731382
3,4,600,601,618,619,40TH ST E,A40,NaN,NaN,55407,...,PORTLAND AVE,NaN,NaN,OAKLAND AVE,NaN,NaN,4de4c732-6273-4758-926a-fb5480518a57,1,1.0,140.356403
4,5,3198,3199,3100,3101,46TH AVE S,A40,NaN,NaN,55406,...,32ND ST E,NaN,NaN,31ST ST E,NaN,NaN,89ca243a-d516-4e54-8a98-13bb4fa0d8ff,1,1.0,271.241016


In [5]:
street = pd.DataFrame(data.StartCenterlineID.unique()).dropna()
dis = pd.DataFrame(np.zeros([len(street),len(street)]))
dis.columns = street.iloc[:,0]

In [18]:
grouped = data.groupby(['StartCenterlineID'])

In [13]:
street['total_dis']= np.zeros(len(street))
street.columns = ['ind','total_dis']

In [16]:
for ID in street.ind:
    street.loc[street.ind==ID,'total_dis'] = grouped.get_group(ID)['TripDistance'].sum()
    street.loc[street.ind==ID,'total_dur'] = grouped.get_group(ID)['TripDuration'].sum()    

In [28]:
n = len(street)

In [36]:
m = Model('battery')
# variables
xvar = {}
for j in range(n):
    xvar[j] = m.addVar(name='y'+str(j),vtype=GRB.BINARY)
for i in range(n):
    for j in range(n):
        xvar[10*i+j] = m.addVar(name='x'+str(i)+','+str(j),ub=1)
for j in range(n):
    xvar[j] = m.addVar(name='z'+str(j))

<gurobi.Model Continuous instance battery: 0 constrs, 0 vars, No parameter changes>